In [ ]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path
%cd "/content/drive/MyDrive/Ex4Files" 
import import_ipynb
import utils

In [ ]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################

# kernel size for the 1D convolution layer

CONV_1D_SIZE = 11  

# number of ResNet blocks for the first ResNet and the kernel size.

RESNET_1_BLOCKS = 3 
RESNET_1_SIZE = (11, 11)

# learning rate and batch size.
LR = 0.001
BATCH = 32 

###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 1  # good start may be 3/5/7
DILATION = [1]
RESNET_2_SIZE = (1,1)  # good start may be (3,3)/(5,5)/(7,7)

# percentage of dropout for the dropout layer

DROPOUT = 0 # good start may be 0.1-0.5

# number of epochs

EPOCHS = 1


In [ ]:
def resnet_1(input_layer, n):
    """
    ResNet layer - input -> Conv2D -> Relu -> Conv2D -> BatchNormalization -> Add -> Relu
    :param input_layer: input layer for the ResNet
    :param n: number of ResNet blocks
    :return: last layer of the ResNet
    """
    for i in range(n):
        conv_layer = layers.Conv2D(32, RESNET_1_SIZE, activation="relu", padding='same')(input_layer)
        conv_layer = layers.Conv2D(32, RESNET_1_SIZE, padding='same')(conv_layer)
        batch_layer = layers.BatchNormalization()(conv_layer)
        add_layer = layers.Add()([batch_layer, input_layer])
        input_layer = layers.Activation("relu")(add_layer)

    return input_layer

In [ ]:
def resnet_2(input_layer, n, dilation):
    """
    Dilated ResNet layer - input -> dilated Conv2D -> Relu -> dilated Conv2D -> BatchNormalization -> Add -> Relu
    :param input_layer: input layer for the ResNet
    :param n: number of ResNet repetitions
    :param dilation: list of int (for example [1,2,4]), dilation list for each repetition.
    :return: last layer of the ResNet
    """


In [ ]:
def distance_layer(input_layer):
    """
    distance output layer- input -> Conv2D -> Conv2D -> Relu -> combine with transpose
    :param input_layer: keras layer
    :return: distance output layer (32*32*1) with name='distances'
    """
    distances = layers.Conv2D(4, (5,5), padding="same",activation="elu")(input_layer)
    distances = layers.Conv2D(1, (5,5), activation="relu",padding="same")(distances)
    distance_t = layers.Permute((2, 1, 3))(distances)
    distances = layers.Add(name="distances")([0.5 * distances, 0.5 * distance_t])  # for symmetry

    return distances

In [ ]:
def theta_layer(input_layer):
    """
    theta output layer- input -> Conv2D -> Conv2D -> tanh
    :param input_layer: keras layer
    :return: theta output layer (32*32*2) with name='thetas'
    """
    thetas = layers.Conv2D(4, (5,5), padding="same", activation="elu")(input_layer)
    thetas = layers.Conv2D(2, (5,5), activation="tanh", padding="same",name="thetas")(thetas)

    return thetas


In [ ]:
def omega_layer(input_layer):
    """
    omega output layer- input -> Conv2D -> Conv2D -> tanh -> combine with transpose
    :param input_layer: keras layer
    :return: omega output layer (32*32*2) with name='omegas'
    """

In [ ]:
def phi_layer(input_layer):
    """
    phi output layer- input -> Conv2D -> Conv2D -> tanh
    :param input_layer: keras layer
    :return: phi output layer (32*32*2) with name='phis'
    """

In [ ]:
def build_network():
    """
    builds the neural network architecture as shown in the exercise.
    :return: Keras Model
    """
    # input size 32*21
    input_layer = tf.keras.Input(shape=(32, 21), name="InputLayer")

    # Conv1D -> size = 32*32
    conv1d_layer = layers.Conv1D(32, CONV_1D_SIZE, padding='same')(input_layer)
    # reshape, so we can use Conv2D -> size = 32*32*1
    reshape_layer = layers.Reshape((32, 32, 1))(conv1d_layer)

    # Conv2D -> size = 32*32*32
    conv2d_layer = layers.Conv2D(32, RESNET_1_SIZE, padding='same')(reshape_layer)

    # first ResNet -> size = 32*32*32
    resnet_layer = resnet_1(conv2d_layer, RESNET_1_BLOCKS)

    # Conv2D -> size = 32*32*64
    conv2d_layer = layers.Conv2D(64, RESNET_2_SIZE, padding="same")(resnet_layer)

    # second res block: dilated_2d_conv -> relu -> dilated_2d_conv -> batch_normalization -> add -> relu  (size = 32*32*64)
    resnet_layer = resnet_2(conv2d_layer, RESNET_2_BLOCKS, DILATION)

    # dropout -> size = 32*32*64
    dropout_layer = layers.Dropout(DROPOUT)(resnet_layer)

    # distance output -> size = 32*32*1
    distances = distance_layer(dropout_layer)
    # omega output -> size = 32*32*2
    omegas = omega_layer(dropout_layer)
    # theta output -> size = 32*32*2
    thetas = theta_layer(dropout_layer)
    # phi output -> size = 32*32*2
    phis = phi_layer(dropout_layer)

    return tf.keras.Model(input_layer, [distances, omegas, thetas, phis], name="my_network")


In [ ]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 4, figsize=(15,3))
    axes[0].plot(history.history['distances_loss'], label='Training loss')
    axes[0].plot(history.history['val_distances_loss'], label='Validation loss')
    axes[0].legend()
    axes[0].set_title("Distance loss")

    axes[1].plot(history.history['omegas_loss'], label='Training loss')
    axes[1].plot(history.history['val_omegas_loss'], label='Validation loss')
    axes[1].legend()
    axes[1].set_title("Omega loss")

    axes[2].plot(history.history['thetas_loss'], label='Training loss')
    axes[2].plot(history.history['val_thetas_loss'], label='Validation loss')
    axes[2].legend()
    axes[2].set_title("Theta loss")

    axes[3].plot(history.history['phis_loss'], label='Training loss')
    axes[3].plot(history.history['val_phis_loss'], label='Validation loss')
    axes[3].legend()
    axes[3].set_title("Phi loss")

    plt.savefig("/content/drive/MyDrive/Ex4Files/model_loss_history")  # TODO: you can change the path here


In [ ]:
def plot_distance_heatmap(true_dist, predicted_dist):
    """
    plots the true and predicted pairwise distances of a nanobody, saves it in 'distance_heatmap.png'
    :param true_dist: true pairwise distance matrix ( n * n * 1)
    :param predicted_dist: predicted pairwise distance matrix ( n * n * 1)
    :return: None
    """
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))
    sns.heatmap(true_dist[:,:,0],
                ax=axes[0], xticklabels=False,
                yticklabels=False, cbar=True,
                cbar_kws={"shrink": 1, "pad": 0.03, "fraction": 0.04},
                linewidth=0.003, linecolor="#222", cmap="rocket")
    sns.heatmap(predicted_dist[:,:,0],
                ax=axes[1], xticklabels=False,
                yticklabels=False, cbar=True,
                cbar_kws={"shrink": 1, "pad": 0.03, "fraction": 0.04},
                linewidth=0.003, linecolor="#222", cmap="rocket")

    axes[0].set_title("True Distances", fontdict={'fontsize': 18})
    axes[1].set_title("Predicted Distances",fontdict={'fontsize': 18})

    plt.tight_layout()
    plt.savefig("/content/drive/MyDrive/Ex4Files/distance_heatmap", dpi=200)  # TODO: you can change the path here



In [ ]:
if __name__ == '__main__':

    # model = build_network()


    # you can load here your input and output data

    # X = numpy array of size 2141*32*21 of all the data input
    # dist = numpy array of size 2141*32*32*1 of all the data distances
    # omega = numpy array of size 2141*32*32*2 of all the data omegas
    # theta = numpy array of size 2141*32*32*2 of all the data thetas
    # phi = numpy array of size 2141*32*32*2 of all the data phis


    # split into validation and test sets as you like


    # b)
    #  compile model using Adam optimizer (with learning rate of your choice) and MSE loss.

    # c)
    # fit model (use EPOCH for epoch parameter and BATCH for batch_size parameter)

    # d)
    # save model

    # generate and plot the distance constraints of Nb 5jds using the network you built 
